# 00 - Environment Setup

This is the notebook that sets up the GCP project for the other notebooks in this repository.  Based on the [`Readme.md`](https://github.com/statmike/vertex-ai-mlops/blob/main/readme.md), you already have this repository of notebooks pulled as a local resource in your Vertex AI Workbench based notebook instance.

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:
<p align="center" width="100%" width="100%"><center><a href="https://youtu.be/pnQ5Rv4ZQfo" target="_blank" rel="noopener noreferrer"><img src="../architectures/thumbnails/playbutton/00.png" width="40%"></a></center></p>

### Conceptual Flow & Workflow
<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/00_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/00_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
# project = !gcloud config get-value project
PROJECT_ID = 'gcp-demo1-362917'
PROJECT_ID

'gcp-demo1-362917'

In [2]:
REGION = 'us-central1'

DATANAME = 'fraud'

# Data source for this series of notebooks: Described in notebook 01
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

>**Note**<p>This repository is set to use a BQ_SOURCE table from the `bigquery-public-data` project.  More information on this BigQuery project [here](https://cloud.google.com/bigquery/public-data)</p>

packages:

In [3]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

clients:

In [42]:
# use project_id to name the bucket
BUCKET = PROJECT_ID
# create storage bucket
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [43]:
BUCKET

'gcp-demo1-362917'

---
## Create Storage Bucket
Check to see if bucket already exist and create if missing:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [44]:
gcs = storage.Client(project = PROJECT_ID)
gcs

In [45]:
# Create Storage Bucket need use project_id, not just project name
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Bucket already exist: gcp-demo1-362917


---
## Store Project Data in the Storage Bucket
Check to see if the export exist and create if not:
- export from bigquery table to GCS bucket as CSV
    - the table is referenced in the `BQ_SOURCE` variable at the top of this notebook
- [Exporting Table Data](https://cloud.google.com/bigquery/docs/exporting-data#python)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [46]:
# bq = bigquery.Client(project = PROJECT_ID)
# destination = f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv"
# source = bigquery.TableReference.from_string(BQ_SOURCE)
# extract = bq.extract_table(source,destination)
# =======================================================================
# gcs = storage.Client(project = PROJECT_ID)

# DATANAME = 'fraud'

# BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

file = f"{DATANAME}/data/{DATANAME}.csv"

In [47]:
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

Creating the export ...
Exported the table to: gs://gcp-demo1-362917/fraud/data/fraud.csv


---
## Install KFP
If you get an error after a step, rerun it.  The dependecies sometimes resolve.
- [Install the Kubeflow Pipelines SDK](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [4]:
!pip install kfp -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx 0.26.3 requires absl-py<0.11,>=0.9, but you have absl-py 1.2.0 which is incompatible.
tfx 0.26.3 requires attrs<21,>=19.3.0, but you have attrs 22.1.0 which is incompatible.
tfx 0.26.3 requires click<8,>=7, but you have click 8.1.3 which is incompatible.
tfx 0.26.3 requires docker<5,>=4.1, but you have docker 5.0.3 which is incompatible.
tfx 0.26.3 requires pyarrow<0.18,>=0.17, but you have pyarrow 9.0.0 which is incompatible.
tfx-bsl 0.26.1 requires absl-py<0.11,>=0.9, but you have absl-py 1.2.0 which is incompatible.
tfx-bsl 0.26.1 requires pyarrow<0.18,>=0.17, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-transform 0.26.0 requires absl-py<0.11,>=0.9, but you have absl-py 1.2.0 which is incompatible.
tensorflow-transform 0.26.0 requires pyarrow<0.18,>=0.17, but you have pyarrow 9.0.0 which is 

In [5]:
!pip install google-cloud-pipeline-components -U -q

---
## Installs For Specific Notebooks

10 - Plotly used for visualizations
- [Getting Started with Plotly in Python](https://plotly.com/python/getting-started/)

In [6]:
!pip install plotly -q

---
## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)

In [7]:
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q